<a href="https://colab.research.google.com/github/QuanNguyen28/Barefoot/blob/main/RAG_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas faiss-cpu sentence-transformers openai

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Đọc và tiền xử lý dữ liệu tin tức từ các nguồn

In [3]:
import pandas as pd
import numpy as np
import re
import json
from sentence_transformers import SentenceTransformer
import faiss
import openai
import matplotlib.pyplot as plt
import seaborn as sns

def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'<[^>]+>', '', str(text))
    text = re.sub(r'<[^>]+>|[\*\#\@]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_ticker(text):
    text = str(text).upper()
    # Thêm các pattern phức tạp hơn và xử lý viết tắt
    patterns = [
        r'\b(FPT|CMG)\b',
        r'\b(FPT\d*[A-Z]*)\b',
        r'\b(CMG\d*[A-Z]*)\b'
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(0)
    return 'UNKNOWN'

def preprocess_news(df, source_label):
    title_col = 'title' if 'title' in df.columns else df.columns[0]
    content_col = 'summary' if 'summary' in df.columns else df.columns[1]

    # Cột ngày
    if 'date' in df.columns:
        date_col = 'date'
    else:
        possible_date = [col for col in df.columns if "date" in col.lower() or "ngày" in col.lower()]
        if possible_date:
            date_col = possible_date[0]
        else:
            raise ValueError(f"Không tìm thấy cột ngày trong DataFrame {source_label}")

    # Làm sạch
    df['title'] = df[title_col].apply(clean_text)
    df['content'] = df[content_col].apply(clean_text)
    df['text'] = df['title'] + ". " + df['content']

    # Parse ngày: mặc định mm/dd/yyyy → dayfirst=False
    df['date'] = pd.to_datetime(df[date_col], errors='coerce', dayfirst=False)

    df['source'] = source_label
    df['record_date'] = df['date']

    ticker_col = 'ticker' if 'ticker' in df.columns else None

    # Nếu có ticker thì dùng, không thì trích
    if ticker_col:
        df['ticker'] = df[ticker_col]
    else:
        df['ticker'] = df['text'].apply(extract_ticker)

    return df[['record_date', 'date', 'ticker', 'text', 'source']]

def process_divided(df, source_label):

    # Đầu tiên, chuẩn hóa tên cột về dạng dễ xử lý nếu cần
    df.columns = df.columns.str.strip().str.lower()

    # Đổi tên cho dễ code
    rename_mapping = {
        'exchange': 'exchange',
        'ex-dividend date': 'ex_dividend_date',
        'record date': 'record_date',
        'execution date': 'execution_date',
        'event content': 'event_content',
        'event type': 'event_type'
    }
    df = df.rename(columns=rename_mapping)

    # Tạo cột mới gộp thông tin
    def combine_event_info(row):
        parts = []
        parts.append(f"Sàn giao dịch: {row['exchange'] if pd.notna(row['exchange']) and row['exchange'] else 'UNKNOWN'}.")
        parts.append(f"Ngày giao dịch không hưởng quyền: {row['ex_dividend_date'] if pd.notna(row['ex_dividend_date']) and row['ex_dividend_date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày chốt danh sách: {row['record_date'] if pd.notna(row['record_date']) and row['record_date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày thực hiện: {row['execution_date'] if pd.notna(row['execution_date']) and row['execution_date'] else 'UNKNOWN'}.")
        parts.append(f"Nội dung sự kiện: {row['event_content'] if pd.notna(row['event_content']) and row['event_content'] else 'UNKNOWN'}.")
        parts.append(f"Loại sự kiện: {row['event_type'] if pd.notna(row['event_type']) and row['event_type'] else 'UNKNOWN'}.")
        return " ".join(parts)

    df['text'] = df.apply(combine_event_info, axis=1)
    df['date'] = pd.to_datetime(df['execution_date'], errors='coerce', dayfirst=True)
    df['record_date'] = pd.to_datetime(df['record_date'], errors='coerce', dayfirst=True)

    df['source'] = source_label
    df['ticker'] = df['stockid'] if 'stockid' in df.columns else 'UNKNOWN'
    # df[['record_date', 'date', 'ticker', 'text', 'source']]

    return df[['record_date', 'date', 'ticker', 'text', 'source']]

def process_shareholder(df, source_label):

    # Đầu tiên, chuẩn hóa tên cột về dạng dễ xử lý nếu cần
    df.columns = df.columns.str.strip().str.lower()

    # Đổi tên cho dễ code
    rename_mapping = {
        'exchange': 'exchange',
        'ex-rights date': 'ex_rights_date',
        'record date': 'record_date',
        'execution date': 'execution_date',
        'event type': 'event_type'
    }
    df = df.rename(columns=rename_mapping)

    # Tạo cột mới gộp thông tin
    def combine_event_info(row):
        parts = []
        parts.append(f"Sàn giao dịch: {row['exchange'] if pd.notna(row['exchange']) and row['exchange'] else 'UNKNOWN'}.")
        parts.append(f"Ngày giao dịch không hưởng quyền: {row['ex_rights_date'] if pd.notna(row['ex_rights_date']) and row['ex_rights_date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày chốt danh sách: {row['record_date'] if pd.notna(row['record_date']) and row['record_date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày thực hiện: {row['execution_date'] if pd.notna(row['execution_date']) and row['execution_date'] else 'UNKNOWN'}.")
        parts.append(f"Loại sự kiện: {row['event_type'] if pd.notna(row['event_type']) and row['event_type'] else 'UNKNOWN'}.")
        return " ".join(parts)


    df['text'] = df.apply(combine_event_info, axis=1)
    df['date'] = pd.to_datetime(df['execution_date'], errors='coerce', dayfirst=True)
    df['record_date'] = pd.to_datetime(df['record_date'], errors='coerce', dayfirst=True)
    df['source'] = source_label
    df['ticker'] = df['stockid'] if 'stockid' in df.columns else 'UNKNOWN'
    # df[['record_date', 'date', 'ticker', 'text', 'source']]

    return df[['record_date', 'date', 'ticker', 'text', 'source']]


def process_internal(df, source_label):
    # Chuẩn hóa tên cột
    df.columns = df.columns.str.strip().str.lower()

    # Các cột bạn muốn gộp
    columns_to_combine = [
        'transaction type', 'executor name', 'executor position', 'related person name',
        'related person position', 'relation', 'before transaction volume', 'before transaction percentage',
        'registered transaction volume', 'registered from date', 'registered to date',
        'executed transaction volume', 'executed from date', 'executed to date',
        'after transaction volume', 'after transaction percentage'
    ]

    for col in columns_to_combine:
        if col in df.columns:
            df[col] = df[col].apply(clean_text)

    # Hàm gộp thành text
    def combine_fields(row):
        parts = []
        parts.append(f"Loại giao dịch: {row['transaction type'] if pd.notna(row['transaction type']) and row['transaction type'] else 'UNKNOWN'}.")
        parts.append(f"Người thực hiện: {row['executor name'] if pd.notna(row['executor name']) and row['executor name'] else 'UNKNOWN'}.")
        parts.append(f"Chức vụ người thực hiện: {row['executor position'] if pd.notna(row['executor position']) and row['executor position'] else 'UNKNOWN'}.")
        parts.append(f"Người liên quan: {row['related person name'] if pd.notna(row['related person name']) and row['related person name'] else 'UNKNOWN'}.")
        parts.append(f"Chức vụ người liên quan: {row['related person position'] if pd.notna(row['related person position']) and row['related person position'] else 'UNKNOWN'}.")
        parts.append(f"Quan hệ: {row['relation'] if pd.notna(row['relation']) and row['relation'] else 'UNKNOWN'}.")
        parts.append(f"Số lượng trước giao dịch: {row['before transaction volume'] if pd.notna(row['before transaction volume']) and row['before transaction volume'] else 'UNKNOWN'}.")
        parts.append(f"Tỷ lệ trước giao dịch: {row['before transaction percentage'] if pd.notna(row['before transaction percentage']) and row['before transaction percentage'] else 'UNKNOWN'}%.")
        parts.append(f"Số lượng đăng ký: {row['registered transaction volume'] if pd.notna(row['registered transaction volume']) and row['registered transaction volume'] else 'UNKNOWN'}.")
        parts.append(f"Ngày bắt đầu đăng ký: {row['registered from date'] if pd.notna(row['registered from date']) and row['registered from date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày kết thúc đăng ký: {row['registered to date'] if pd.notna(row['registered to date']) and row['registered to date'] else 'UNKNOWN'}.")
        parts.append(f"Số lượng thực tế giao dịch: {row['executed transaction volume'] if pd.notna(row['executed transaction volume']) and row['executed transaction volume'] else 'UNKNOWN'}.")
        parts.append(f"Ngày bắt đầu thực hiện: {row['executed from date'] if pd.notna(row['executed from date']) and row['executed from date'] else 'UNKNOWN'}.")
        parts.append(f"Ngày kết thúc thực hiện: {row['executed to date'] if pd.notna(row['executed to date']) and row['executed to date'] else 'UNKNOWN'}.")
        parts.append(f"Số lượng sau giao dịch: {row['after transaction volume'] if pd.notna(row['after transaction volume']) and row['after transaction volume'] else 'UNKNOWN'}.")
        parts.append(f"Tỷ lệ sau giao dịch: {row['after transaction percentage'] if pd.notna(row['after transaction percentage']) and row['after transaction percentage'] else 'UNKNOWN'}%.")
        return " ".join(parts)


    # Tạo cột text
    df['text'] = df.apply(combine_fields, axis=1)
    df['date'] = pd.to_datetime(df['executed to date'], errors='coerce', dayfirst=True)
    df['record_date'] = pd.to_datetime(df['executed from date'], errors='coerce', dayfirst=True)
    df['source'] = source_label
    df['ticker'] = df['stockid'] if 'stockid' in df.columns else 'UNKNOWN'

    return df[['record_date', 'date', 'ticker', 'text', 'source']]



#Đọc file từ các nguồn
df_cafef = pd.read_excel("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/Data processed/CafeF_News_FPT_CMG.xlsx")
df_dividend = pd.read_csv("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/Data processed/3.2 (live & his) news_dividend_issue (FPT_CMG)_processed.csv")
df_shareholder = pd.read_csv("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/3.3 (live & his) news_shareholder_meeting (FPT_CMG)_processed.csv")
df_internal = pd.read_csv("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/3.4 (live & his) news_internal_transactions (FPT_CMG)_processed.csv")

#Tiền xử lý từng DataFrame
df_cafef_clean = preprocess_news(df_cafef, "cafef").fillna("UNKNOWN")
df_dividend_clean = process_divided(df_dividend, "dividend").fillna("UNKNOWN")
df_shareholder_clean = process_shareholder(df_shareholder, "shareholder").fillna("UNKNOWN")
df_internal_clean = process_internal(df_internal, "internal").fillna("UNKNOWN")

# Gộp
df_all_news = pd.concat([
    df_cafef_clean, df_dividend_clean, df_shareholder_clean, df_internal_clean
], ignore_index=True)


#Hợp nhất tất cả dữ liệu tin tức
df_all_news = pd.concat([
    df_cafef_clean, df_dividend_clean, df_shareholder_clean, df_internal_clean
], ignore_index=True)

print(df_all_news)
# save
df_all_news.to_csv("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/Data processed/all_news.csv", index=False)

             record_date                 date ticker  \
0    2025-03-12 00:00:00  2025-03-12 00:00:00    FPT   
1    2025-03-11 00:00:00  2025-03-11 00:00:00    FPT   
2    2025-03-11 00:00:00  2025-03-11 00:00:00    FPT   
3    2025-03-11 00:00:00  2025-03-11 00:00:00    FPT   
4    2025-03-11 00:00:00  2025-03-11 00:00:00    FPT   
..                   ...                  ...    ...   
982  2023-05-22 00:00:00  2023-06-20 00:00:00    CMG   
983  2023-04-26 00:00:00  2023-05-25 00:00:00    CMG   
984  2023-03-23 00:00:00  2023-04-21 00:00:00    CMG   
985  2023-03-15 00:00:00  2023-04-13 00:00:00    CMG   
986  2023-03-14 00:00:00  2023-03-14 00:00:00    CMG   

                                                  text    source  
0    Phiên 12/3: Khối ngoại bán chiến biến hơn 900 ...     cafef  
1    Chứng minh ngày mai (12-3): VN-Index tiếp tục ...     cafef  
2    FPT "Bắt tay" Tỉnh Bắc Giang phát triển toàn d...     cafef  
3    CTCK tự doanh không mong đợi trở lại "gom" một...     

# Chunking

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

# Thiết lập bộ chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,         # Số ký tự mục tiêu mỗi đoạn (~300-500 tokens)
    chunk_overlap=50,       # Phần overlap giữa các chunk
    separators=["\n\n", "\n", ".", " ", ""],  # Ưu tiên tách theo đoạn > câu > từ
)

# Áp dụng chunking vào từng dòng trong df_all_news
chunks = []

for idx, row in df_all_news.iterrows():
    split_texts = text_splitter.split_text(row['text'])
    for chunk_text in split_texts:
        chunks.append({
            "text": chunk_text,
            "ticker": row['ticker'],
            "record_date": row['record_date'],
            "date": row['date'],
            "source": row['source']
        })

df_chunks = pd.DataFrame(chunks)

# Kết quả
print(f"Số lượng chunk tạo ra: {len(df_chunks)}")
print(df_chunks.head())
df_chunks.to_csv("/content/drive/MyDrive/Barefoots/Vòng 2/CSV/DATA EXPLORER CONTEST/News - FPT & CMG/Data processed/chunks.csv", index=False)


Số lượng chunk tạo ra: 4265
                                                text ticker  \
0  Phiên 12/3: Khối ngoại bán chiến biến hơn 900 ...    FPT   
1  . Hoạt động giao dịch của khối ngoại: Khối ngo...    FPT   
2  . Tổng quan HNX và UPCOM: Trên HNX, khối ngoại...    FPT   
3  . Nhìn chung, trong khi VN-Index cho thấy khả ...    FPT   
4  Chứng minh ngày mai (12-3): VN-Index tiếp tục ...    FPT   

           record_date                 date source  
0  2025-03-12 00:00:00  2025-03-12 00:00:00  cafef  
1  2025-03-12 00:00:00  2025-03-12 00:00:00  cafef  
2  2025-03-12 00:00:00  2025-03-12 00:00:00  cafef  
3  2025-03-12 00:00:00  2025-03-12 00:00:00  cafef  
4  2025-03-11 00:00:00  2025-03-11 00:00:00  cafef  


#embedding

In [5]:
# embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# embeddings = embedding_model.encode(df_chunks['text'].tolist(), show_progress_bar=True)
# print("Embedding shape:", embeddings.shape)


In [21]:
# login vào huggingface
from huggingface_hub import login
login()

In [23]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from tqdm import tqdm

# Tải mô hình PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Hàm mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # (batch_size, seq_len, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# Hàm encode toàn bộ df_chunks['text']
def encode_phobert(texts):
    embeddings = []
    for text in tqdm(texts, desc="Encoding with PhoBERT"):
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt', max_length=512)
        # Move encoded_input to the same device as the model
        encoded_input = encoded_input.to(device) # This line has been added to move the input to the GPU
        with torch.no_grad():
            model_output = model(**encoded_input)
        sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        embeddings.append(sentence_embedding.squeeze(0).cpu().numpy())
    return np.vstack(embeddings)

# Dùng để embedding
texts = df_chunks['text'].tolist()
embeddings = encode_phobert(texts)

print("Embedding shape:", embeddings.shape)

Encoding with PhoBERT: 100%|██████████| 4265/4265 [00:47<00:00, 90.24it/s]

Embedding shape: (4265, 768)


#Lưu trữ vào FAISS

In [24]:
import faiss
import numpy as np

# Chuẩn bị dimension
dimension = embeddings.shape[1]

# Khởi tạo FAISS Index
index = faiss.IndexFlatIP(dimension)   # Dùng Inner Product thay vì L2 (vì đã normalize rồi)

# Normalize embedding trước khi add
faiss.normalize_L2(embeddings)

# Add vào FAISS index
index.add(embeddings)

print("FAISS index có số vector:", index.ntotal)

# Gán mapping index vào df_chunks
df_chunks['embedding_index'] = range(len(df_chunks))


FAISS index có số vector: 4265


#Query Transformation

In [25]:
def retrieve(query, top_k=5):
    """
    Nhận truy vấn dạng chuỗi,
    tạo vector embedding cho truy vấn bằng PhoBERT,
    normalize query vector,
    tìm kiếm top_k vector (chunk) gần nhất từ FAISS index,
    trả về danh sách văn bản.
    """
    # Tokenize và encode query bằng PhoBERT
    encoded_input = tokenizer(query, padding=True, truncation=True, return_tensors='pt', max_length=512)

    # Move encoded_input to the same device as the model
    encoded_input = encoded_input.to(device) # Move encoded_input to GPU if available

    with torch.no_grad():
        model_output = model(**encoded_input)

    # Mean pooling
    query_embedding = mean_pooling(model_output, encoded_input['attention_mask']).cpu().numpy()

    # Normalize query vector
    faiss.normalize_L2(query_embedding)

    # Search trong FAISS
    D, I = index.search(query_embedding, top_k)

    # Lấy text chunks tương ứng
    results = df_chunks.iloc[I[0]]['text'].tolist()
    return results

In [26]:
# prompt: lấy kết quả từ truy vấn retrieve và chuyển chúng thành một chuỗi văn bản có thể hiển thị

def retrieve_and_format(query, top_k=10):
    results = retrieve(query, top_k)
    formatted_results = ""
    for i, result in enumerate(results):
        formatted_results += f"Thông tin {i+1}:\n{result}\n"
    return formatted_results

In [27]:
query = "FPT có điểm gì nổi bật năm 2025?"
source_information = retrieve_and_format(query)
print("Phản hồi từ hệ thống RAG:")
print(source_information)

Phản hồi từ hệ thống RAG:
Thông tin 1:
. Ngành cơ điện dự kiến phục hồi với mức tăng trưởng dự báo 21% trong giai đoạn 2025-2028. Gemadept Corporation (GMD): Dự báo sản lượng thông qua cảng tăng trưởng trên 40% vào năm 2024. Các diễn biến mới dự kiến sẽ thúc đẩy tăng trưởng trung và dài hạn sau năm 2026. Hòa Phát Group (HPG): Dự kiến lợi nhuận tăng trưởng 26% vào năm 2025, được hỗ trợ bởi giá thép trong nước ổn định và tiêu dùng được cải thiện
Thông tin 2:
Nhóm công ty chứng khoán 'lăng xê' thuộc ngành nào trong năm 2025?. Tóm tắt thị trường (28/01/2025) Dự báo VN-Index: KBSV Research dự báo VN-Index sẽ đạt 1.460 điểm vào cuối năm 2025, tương ứng với mức tăng 16,7% trong EPS trung bình của các công ty niêm yết trên HOSE, với tỷ lệ P/E mục tiêu là 14,6, thấp hơn mức trung bình 10 năm là 16,6
Thông tin 3:
Năm rực rỡ của cổ phiếu “họ” FPT, vẫn còn một cái tên “lạc lõng”. Tóm tắt thị trường - Ngày 15 tháng 02 năm 2024 Thị trường chứng khoán Việt Nam đối mặt với nhiều thách thức trong năm 2

In [28]:
def format_prompt(query, context):
    return f"""<s>[INST]
Bạn là chuyên gia phân tích tài chính. Hãy trả lời câu hỏi dựa trên thông tin sau:
Câu hỏi: {query}

Thông tin tham khảo:
{context}

Yêu cầu:
- Trả lời ngắn gọn, tự nhiên (không dấu đầu dòng)
- Luôn ghi rõ ngày, nguồn thông tin nếu có
- Nếu không đủ thông tin để kết luận, hãy nói rõ
[/INST]
"""
print(format_prompt(query, source_information))

<s>[INST]
Bạn là chuyên gia phân tích tài chính. Hãy trả lời câu hỏi dựa trên thông tin sau:
Câu hỏi: FPT có điểm gì nổi bật năm 2025?

Thông tin tham khảo:
Thông tin 1:
. Ngành cơ điện dự kiến phục hồi với mức tăng trưởng dự báo 21% trong giai đoạn 2025-2028. Gemadept Corporation (GMD): Dự báo sản lượng thông qua cảng tăng trưởng trên 40% vào năm 2024. Các diễn biến mới dự kiến sẽ thúc đẩy tăng trưởng trung và dài hạn sau năm 2026. Hòa Phát Group (HPG): Dự kiến lợi nhuận tăng trưởng 26% vào năm 2025, được hỗ trợ bởi giá thép trong nước ổn định và tiêu dùng được cải thiện
Thông tin 2:
Nhóm công ty chứng khoán 'lăng xê' thuộc ngành nào trong năm 2025?. Tóm tắt thị trường (28/01/2025) Dự báo VN-Index: KBSV Research dự báo VN-Index sẽ đạt 1.460 điểm vào cuối năm 2025, tương ứng với mức tăng 16,7% trong EPS trung bình của các công ty niêm yết trên HOSE, với tỷ lệ P/E mục tiêu là 14,6, thấp hơn mức trung bình 10 năm là 16,6
Thông tin 3:
Năm rực rỡ của cổ phiếu “họ” FPT, vẫn còn một cái tên 

#Triển khai

In [29]:
pip install transformers

In [30]:
!pip install tiktoken

In [31]:
!pip install transformers_stream_generator

In [32]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")


In [33]:
def generate_response_from_gemma(query, context_chunks, max_new_tokens=500):
    prompt = format_prompt(query, context_chunks)

    # Tokenize + đưa vào GPU
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode kết quả
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tách phần trả lời sau <start_of_turn>model
    if "<start_of_turn>model" in decoded:
        return decoded.split("<start_of_turn>model")[-1].strip()
    return decoded.strip()


In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    #device_map="auto", # Comment out device_map="auto"
    trust_remote_code=True,
    # Added: offload_folder for disk offloading
    offload_folder="offload", # Specify a folder to offload to
    # Added: low_cpu_mem_usage for optimized CPU RAM usage
    low_cpu_mem_usage=True
)

# Explicitly move the model to the GPU if available
if torch.cuda.is_available():
    model.cuda() # or model.to('cuda')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [39]:
def generate_response_from_mistral(query, context_chunks, max_new_tokens=512):
    prompt = format_prompt(query, context_chunks)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    # Change: Move the entire model to the desired device
    if torch.cuda.is_available():
        model.to('cuda')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        # Removed attention_mask from the separate argument
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            # attention_mask=inputs.get("attention_mask", None) # removed this line
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Loại bỏ phần prompt nếu cần
    if "[/INST]" in response:
        return response.split("[/INST]")[-1].strip()
    return response.strip()

In [40]:
context_chunks = format_prompt(query, source_information)
answer = generate_response_from_mistral(query, context_chunks, 512)
print("🧠 Câu trả lời từ Gemma:")
print(answer)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 16.12 MiB is free. Process 42336 has 14.72 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 30.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)